In [ ]:
#coding: utf-8
import numpy as np
import pandas as pd
import re
import jieba
import os
import time
import jieba  #处理中文
import numpy as np
from progressbar import *
start1 =time.clock()

In [ ]:
def obatin_news_from_csv(path_csv):
    old_text=pd.read_csv(path_csv)
    drop_index_na=[]
    for i in range(old_text.shape[0]):
        j=old_text['微博正文'][i]
        if ("抱歉" in j or len(j)<40):
            drop_index_na.append(i)
    text=old_text.drop(drop_index_na)
    text=text.reset_index(drop=True)
    return text

import re
def find_title(text):
    text["标题"]=[-99 for i in range(text.shape[0])]
    title_p=re.compile(r".*\【.*\】")
    progress = ProgressBar()
    for i in progress(range(len(text["微博正文"]))):
        title_=re.match(title_p,text["微博正文"][i])
        if title_:
            text["标题"][i]=title_[0]
        text["微博正文"][i]=re.sub(".*\【.*\】","",text["微博正文"][i])
        time.sleep(0.1)
    notitle_index=[]
    for i in range(text.shape[0]):
        if text["标题"][i]==-99:
            notitle_index.append(i)
    text=text.drop(notitle_index)
    text=text.reset_index(drop=True)
    return text

In [ ]:
#载入词典和停用词
jieba.load_userdict(r".\自定义词典.txt")
stopwords = [line.strip() for line in open(r".\停用词库.txt",encoding='UTF-8').readlines()]
#清理内容
def clean_text_content(text_content,cut_all=False):
    text_content = re.sub(r"。", ".", text_content)
    text_content = re.sub(r"，", ",", text_content)
    text_content = re.sub(r"“", "'", text_content)
    text_content = re.sub(r"”", "'", text_content)
    text_content = re.sub(r"…", ".", text_content)
    text_content = re.sub(r"@", "@", text_content)
    text_content = re.sub(r" ", " ", text_content)
    text_content = re.sub(r"！", "!", text_content)
    text_content = re.sub(r"？", "?", text_content)
    text_content = re.sub(r"：", ":", text_content)
    text_content = re.sub(r"）", ")", text_content)
    text_content = re.sub(r"（", "(", text_content)
    text_content = re.sub(r"(\d+年)*(\d+月)*(\d+[日])", "", text_content) #日期
    text_content = re.sub(r"\d+[年月日天号人名时例名省市区县院]", "", text_content) 
    text_content = re.sub(r"[第]*[零一二三四五六七八九百千万]+[年月日天号人名时例名省市区县院名例周月年]*", "", text_content) 
    text_content = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text_content) #时间
    text_content = re.sub(r"^[-+]?[0-9]+(\.)?[0-9]*$", "", text_content) #数字
    text_content = re.sub(r"@\S*\s", " ", text_content) #@小央视频
    text_content = re.sub(r"\[.+\]", "", text_content) #[组图共2张]和[加油]
    text_content = re.sub(r"\(.*\)", "", text_content) #（环球网）
    
    text_content = re.sub(r"\W+\w*(视频)\s+", "", text_content) #小央视频的秒拍视频
    #de这个bug花费我一晚上时间！！因为一开始写\W+\S*(视频)\s+的话，\S会把,.等也匹配进去，就变成从第一个.匹配了。
    #但是\w是"匹配特殊字符，即非字母、非数字、非汉字、非_"，即不会匹配,.
    #以一下几个做测试：
    #"戳视频↓星辰浩瀚，2020，我们共同努力！新华视点的秒拍视频 @新华视点  "
    #"祝你们每天好“星晴”@iPanda熊猫频道 iPanda熊猫频道的微博视频  "
    #"据悉。日本的独居户已超过三分之一。预计在2040年将达到40%。（全球视频大魔王）全球视频大魔王的微博视频    "

    text_content = re.sub(r"#", "", text_content) ##
    text_content = re.sub(r"(http|https)(://t.cn/)[a-zA-Z0-9]+", "", text_content) #网址(微博上的连接都是http://t.cn/.....形式)
    text_content = re.sub(r"转发理由:", "", text_content) 
    text_content = re.sub(r"转发内容:", "", text_content) 
    text_content = re.sub(r"原始用户:.*", "", text_content) 
    
    word_list=jieba.lcut(text_content,cut_all=cut_all)
    word_list_len=len(word_list)
    i=0
    while i<word_list_len:
        j=word_list[i]
        if j in stopwords or not('\u4e00'<=j<='\u9fff') or len(j)<2:
            word_list.pop(i)
            word_list_len=word_list_len-1
        elif (j=="中方" or j=="我国"):
            word_list[i]="中国"
        elif j=="美方":
            word_list[i]="美国"
        elif j=="贸易战":
            word_list[i]="贸易" 
        else:
            i=i+1
    if " " in word_list:
        word_list.remove(" ")
    return word_list#text_content 

In [ ]:
import re
a=[
    "./数据集2-五个分类/中国农业新闻网/2106046392.csv",
    "./数据集2-五个分类/新浪教育/1225314032.csv",
    "./数据集2-五个分类/新浪财经/1638782947.csv",
    "./数据集2-五个分类/新华体育/1731169267.csv",
    "./数据集2-五个分类/新华国际/2641686425.csv"
]

text_one_news_list=[]
dict_label={"农业":1,"教育":2,"财经":3,"体育":4,"国际":5}

for i in a:
    
    pattern1=re.compile(r"[\u4e00-\u9fa5]*")
    class_name1=pattern1.findall(i)[7]
    class_name=class_name1[len(class_name1)-2:len(class_name1)]
    if class_name=="闻网":
        class_name="农业"
    text_one_news=obatin_news_from_csv(i)
    text_one_news=find_title(text_one_news)
    text_one_news['分类']=[dict_label[class_name] for j in range(text_one_news.shape[0])]
    
    index_text_class=["微博正文","标题","分类"]
    text_one_news_1=text_one_news[index_text_class]
    
    text_one_news_1=text_one_news_1.drop_duplicates()
    text_one_news_list.append(text_one_news_1)

text_one_news_list2=[]
text_one_news_list2.append(text_one_news_list[0].loc[len(text_one_news_list[0])-1000:len(text_one_news_list[0])])
text_one_news_list2.append(text_one_news_list[1].loc[len(text_one_news_list[1])-1000:len(text_one_news_list[1])])
text_one_news_list2.append(text_one_news_list[2].loc[len(text_one_news_list[2])-1000:len(text_one_news_list[2])])
text_one_news_list2.append(text_one_news_list[3].loc[len(text_one_news_list[3])-1000:len(text_one_news_list[3])])
text_one_news_list2.append(text_one_news_list[4].loc[1900:2900])

for i in text_one_news_list:
    print(i.shape[0])

In [ ]:
text=text_one_news_list2[0]
for i in range(1,5):
    text=text.append(text_one_news_list2[i])
text=text.reset_index(drop=True)
print(text.shape[0])
text.head(5)

In [ ]:


start =time.clock()

len_text=text.shape[0]

text["微博正文(无标题切词后)"]=[-99 for i in range(len_text)]
text["标题(切词后)"]=[-99 for i in range(len_text)]

drop_na=[]

progress = ProgressBar()

for u in progress(range(0,len_text)):
    cleaned_content=clean_text_content(text["微博正文"][u])
    text["微博正文(无标题切词后)"][u]=" ".join(cleaned_content)

    cleaned_title=clean_text_content(text["标题"][u],True)
    text["标题(切词后)"][u]=" ".join(cleaned_title)

    if pd.isnull(text.loc[u]).any():
        drop_na.append(u)
        print(u) 
        
        
    time.sleep(0.1)

text=text.drop(drop_na)
text=text.reset_index(drop=True)   

text["微博正文(有标题切词后)"]=[-99 for i in range(len_text)]
progress = ProgressBar()

for u in progress(range(0,len_text)):
    text["微博正文(有标题切词后)"][u]=text["标题(切词后)"][u]+" "+text["微博正文(无标题切词后)"][u]


end = time.clock()

print('Running time: %s Seconds'%(end-start))
text.head()

In [ ]:
outputpath='./test_news_with_title_2.csv'
text.to_csv(outputpath,sep=',',index=False,header=True)

In [ ]:
a=pd.read_csv('./test_news_with_title_2.csv')
a.head()